In [158]:
import tweepy
import datetime
import re
import pandas as pd 
import twint
import nest_asyncio
import spacy
from langdetect import detect
import plotly.express as px
nest_asyncio.apply()

# Extração dos Tweets da Avon

In [2]:
c = twint.Config()

In [4]:
nest_asyncio.apply()

# Extração dos dados no Período Pós BBB (25/01/2021 até 10/02/2021)

In [33]:
# Configure
c = twint.Config()
c.Search = "Avon"
c.Lang = "pt"
c.Pandas = True
c.Since = '2021-01-25'
c.Until = '2021-02-10'
twint.run.Search(c)

Tweets_df = twint.storage.panda.Tweets_df

K4D
1353523778718818304 2021-01-24 23:03:10 -0300 <jamiejimeaca> @kennethsumajit Maging avon ka nalang
1353523526221918210 2021-01-24 23:02:10 -0300 <bevandton> @markneilson111 This is Headlands Beach park and that is the FairPort harbor light house.  You lived west of Cleveland? I live east of Cleveland.  I used to live in Elyria years ago which isn’t too far from Avon. How long has it been since you lived in Ohio?
1353523366762864643 2021-01-24 23:01:32 -0300 <BGsbeautycare> Be the first to get your hands on NEW Lumilayer Rosy Perfect Primer. Get a sneak peek trial size for just $7 with select VDL product purchases.   https://t.co/iv311YeKCE   #avon #vdl #new #lumilayer #rosy #perfectprimer #conditionsapply #brigittesbeautycare  https://t.co/aojc4LiZlD
1353522552648462337 2021-01-24 22:58:18 -0300 <JayTC07> @GriggsKidd41 Avon
1353522525679054848 2021-01-24 22:58:12 -0300 <LucySantos_004> @sophixeu @diarraluv @J_hoprigado @trybeherondale @mariartwork @vcutmi Esse eu comprei das revist

# Extração dos tweets pré BBB (08/01/2021 até 24/01/2021)

In [36]:
# Configure
c = twint.Config()
c.Search = "Avon"
c.Lang = "pt"
c.Pandas = True
c.Since = '2021-01-08'
c.Until = '2021-01-24'
twint.run.Search(c)

pre_bbb = twint.storage.panda.Tweets_df

hor @MelJ_Writes @halo_scot @lifelovelily22 @Johnldeboer @vernsanders @SunIslandMusic @cmramsburg @jgmacleodauthor @Galdrafodr @eleesha @LanaCwrites @barb_avon @ElanaMcdougall @CanvassartS @Wrix2 @Readerhal54 @CarolynRuffles @CordonRoma @DBCarterAuthor @silicasun @InkDisregardit All awesome people here!
1347350199329579008 2021-01-07 22:11:34 -0300 <JackieNestle> Avon Digital Catalog -  https://t.co/xLLJlktjoU
1347350003870650368 2021-01-07 22:10:48 -0300 <HillHamilton> #FF #peopleyoushouldfollow #Writerlift #FollowFriday @AlfonsoTortoise @AliceKayAuthor  @AliceLongaker @aliciabee @AliciaJAuthor @AliGLove @alioop326 @AlixKermes @allison_author @AllisonHawn @AllistarBanks @ban_sidhe @barb_avon @Baron1Irene  https://t.co/ZlFhbDs23p
1347349121376657411 2021-01-07 22:07:17 -0300 <barb_avon> @quineatal Thanks, Matthew.  When I'm mobile, I'll try that.
1347349092805058562 2021-01-07 22:07:10 -0300 <JB__Alcantara> Você conhecer a Avon?  Avontade que eu tenho de te dar uns bjos🤣
13473490113600

# Limpeza dos tweets - Avon

### 1 - Junção dos dados pré e pós BBB

In [49]:
#União dos datasets e marcação do período
Tweets_df['pos_bbb'] = 1
pre_bbb['pos_bbb'] = 0

In [50]:
full_tweets = Tweets_df.append(pre_bbb)

In [52]:
full_tweets.shape

(85710, 39)

Temos ao todo 85710 tweets extraídos e 39 colunas

### 2 - Seleção de features

In [53]:
#listando as colunas
full_tweets.columns

Index(['id', 'conversation_id', 'created_at', 'date', 'timezone', 'place',
       'tweet', 'language', 'hashtags', 'cashtags', 'user_id', 'user_id_str',
       'username', 'name', 'day', 'hour', 'link', 'urls', 'photos', 'video',
       'thumbnail', 'retweet', 'nlikes', 'nreplies', 'nretweets', 'quote_url',
       'search', 'near', 'geo', 'source', 'user_rt_id', 'user_rt',
       'retweet_id', 'reply_to', 'retweet_date', 'translate', 'trans_src',
       'trans_dest', 'pos_bbb'],
      dtype='object')

In [54]:
#avaliando dados nulos
full_tweets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 85710 entries, 0 to 28618
Data columns (total 39 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               85710 non-null  object 
 1   conversation_id  85710 non-null  object 
 2   created_at       85710 non-null  float64
 3   date             85710 non-null  object 
 4   timezone         85710 non-null  object 
 5   place            85710 non-null  object 
 6   tweet            85710 non-null  object 
 7   language         85710 non-null  object 
 8   hashtags         85710 non-null  object 
 9   cashtags         85710 non-null  object 
 10  user_id          85710 non-null  int64  
 11  user_id_str      85710 non-null  object 
 12  username         85710 non-null  object 
 13  name             85710 non-null  object 
 14  day              85710 non-null  int64  
 15  hour             85710 non-null  object 
 16  link             85710 non-null  object 
 17  urls        

In [79]:
columns = ['id', 'date', 'place',
       'tweet', 'language', 'hashtags', 'day', 'hour', 'nlikes', 'nreplies', 'nretweets',
       'search', 'pos_bbb']

In [84]:
#Seleção de colunas relevantes
ft_tweets = full_tweets[columns]

In [86]:
#validação de tweets duplicados
ft_tweets.id.duplicated().value_counts()

False    85710
Name: id, dtype: int64

Não existem tweets com o mesmo ID

In [87]:
#Avaliação da localidade dos tweets
ft_tweets.language.value_counts()

en     42562
pt     28143
es      5574
und     2601
fr      1644
tl       894
it       863
tr       515
in       514
de       462
ja       300
ru       274
nl       191
pl       143
ro       141
et       111
ur        96
fi        83
ht        79
ca        66
no        63
ar        59
hi        58
cs        37
sv        36
cy        25
hu        25
lv        22
sl        16
vi        16
el        15
lt        15
uk        14
ko        11
is         8
bg         7
eu         7
da         5
fa         5
zh         3
th         3
or         1
sr         1
iw         1
am         1
Name: language, dtype: int64

Boa parte dos tweets vieram com o idioma Inglês. Vamos explorar para avaliar se são relevantes ou vieram por engano.

In [90]:
ft_tweets.query("language == 'en'")['tweet']

0                        @asianwrld @FearVentureGGs ohh no
2        Congratulations to all you "People on the Move...
5        New 40 Yard Dash PR’s 2/8/21  Cooper McDannald...
11        The Avon. Stratford. ♥️  https://t.co/bFrgBosUoN
12                     @KingdomElla @equibal_m I claim it🙏
                               ...                        
28611    @PatrickIMcMahon @MarianSAnderson @RunRedMouse...
28612    @barb_avon Me too last week shifting in my cha...
28613                     @timotheehals YER DAD SELLS AVON
28616    @barb_avon Getting old is not for wimps that's...
28617           @barb_avon It's a tie and I feel your pain
Name: tweet, Length: 42562, dtype: object

Vamos tentar detectar se existe algum tweet que foi marcado no idioma inglês e que estão em Português.

In [117]:
ft_tweets.query("language == 'en'")

id                 date place  \
0      1358928392947568641  2021-02-08 20:59:11         
2      1358928123857948673  2021-02-08 20:58:07         
5      1358927649834467329  2021-02-08 20:56:14         
11     1358926652898762755  2021-02-08 20:52:16         
12     1358926614789189634  2021-02-08 20:52:07         
...                    ...                  ...   ...   
28611  1347333570008899591  2021-01-07 21:05:29         
28612  1347333507102765056  2021-01-07 21:05:14         
28613  1347333315574038529  2021-01-07 21:04:29         
28616  1347332615360024577  2021-01-07 21:01:42         
28617  1347332526092800002  2021-01-07 21:01:21         

                                                   tweet language  \
0                      @asianwrld @FearVentureGGs ohh no       en   
2      Congratulations to all you "People on the Move...       en   
5      New 40 Yard Dash PR’s 2/8/21  Cooper McDannald...       en   
11      The Avon. Stratford. ♥️  https://t.co/bFrgBosUoN       en   
12                   @KingdomElla @equibal_m I claim it🙏       en   
...                                                  ...      ...   
28611  @PatrickIMcMahon @MarianSAnderson @RunRedMouse...       en   
28612  @barb_avon Me too last week shifting in my cha...       en   
28613                   @timotheehals YER DAD SELLS AVON       en   
28616  @barb_avon Getting old is not for wimps that's...       en   
28617         @barb_avon It's a tie and I feel your pain       en   

                                                hashtags  day hour  nlikes  \
0                                                     []    1   20       0   
2      [4liferesearch, medifast, jeunesse, beautycoun...    1   20       2   
5                                                     []    1   20       3   
11                                                    []    1   20       3   
12                                                    []    1   20       0   
...                                                  ...  ...  ...     ...   
28611                                                 []    4   21       1   
28612                                                 []    4   21       0   
28613                                                 []    4   21       0   
28616                                                 []    4   21       1   
28617                                                 []    4   21       1   

       nreplies  nretweets search  pos_bbb  
0             1          0   Avon        1  
2             0          1   Avon        1  
5             1          1   Avon        1  
11            0          0   Avon        1  
12            0          0   Avon        1  
...         ...        ...    ...      ...  
28611         1          0   Avon        0  
28612         1          0   Avon        0  
28613         1          0   Avon        0  
28616         1          0   Avon        0  
28617         1          0   Avon        0  

[42562 rows x 13 columns]

In [129]:
def try_detect(tweet):
    try:
        return detect(tweet)
    except:
        return 'not_detected'
en_tweets = ft_tweets.query("language == 'en'")
en_tweets['detected_lang'] = en_tweets.apply(lambda row : try_detect(row['tweet']),axis=1)

In [132]:
en_tweets.detected_lang.value_counts()

en              39565
de                601
nl                385
no                249
tl                225
af                202
fr                179
so                146
id                121
it                111
sv                109
et                106
ca                105
fi                 73
da                 66
cy                 52
sl                 36
tr                 36
ro                 32
es                 32
vi                 29
hu                 24
pt                 23
hr                 12
pl                 11
sw                  8
cs                  6
sk                  5
lt                  4
sq                  4
lv                  3
not_detected        2
Name: detected_lang, dtype: int64

In [133]:
en_tweets.query("detected_lang == 'pt'")

id                 date place  \
3263   1358709564779225088  2021-02-08 06:29:38         
3654   1358656678665871361  2021-02-08 02:59:29         
3895   1358620310031900672  2021-02-08 00:34:58         
13210  1357529378591289346  2021-02-05 00:20:00         
14083  1357424660485447694  2021-02-04 17:23:53         
18139  1356946044538523654  2021-02-03 09:42:02         
19647  1356732399464423424  2021-02-02 19:33:05         
19936  1356694199438041089  2021-02-02 17:01:17         
20146  1356674574474223618  2021-02-02 15:43:19         
26522  1355836012136689664  2021-01-31 08:11:10         
32900  1354917350907252740  2021-01-28 19:20:44         
33513  1354858973615300609  2021-01-28 15:28:46         
34656  1354776753898737664  2021-01-28 10:02:03         
43125  1354546450911404035  2021-01-27 18:46:54         
45671  1354512074416812035  2021-01-27 16:30:18         
49319  1354465462764113920  2021-01-27 13:25:05         
53588  1354040408196718594  2021-01-26 09:16:04         
54699  1353878212263211008  2021-01-25 22:31:34         
8351   1351272979666071559  2021-01-18 17:59:18         
9910   1350942235391623168  2021-01-17 20:05:02         
15851  1349785028268584962  2021-01-14 15:26:43         
16523  1349714084716023810  2021-01-14 10:44:48         
21298  1348808436620947456  2021-01-11 22:46:05         

                                                   tweet language  \
3263                                          So dumb 😩😂       en   
3654                                   @cocoem7 Fuck too       en   
3895        @dalidoisa Coach Avon just around da corner.       en   
13210                       mood igual o do fake da avon       en   
14083  @NA0YACAT perfume do frozen da avon  https://t...       en   
18139                  Postcard  https://t.co/dvVUZtkMdS       en   
19647                             @barb_avon So do I🤫😂❤️       en   
19936                @Memeulous THEY HAVE CROSSED A LINE       en   
20146              @npealicia Yeah, or "Pick a decade" 😂       en   
26522  @BCFC_Supporters NO NO NO NO NO NO NO NO NO NO...       en   
32900  O Perfect Red da Gama True Colour da Avon  htt...       en   
33513                                    juntes com avon       en   
34656  #promoção #oferta  Petit Happy Bug - 50ml - Av...       en   
43125  Lumena Halsey Sarah #SouCristaoSouBolsonaro #C...       en   
45671  @vaidesmaiar Avon queria Bubu MakeUp e ganhou ...       en   
49319                           @exfazendeiro makes avon       en   
53588                        @KrllGojira And now so do I       en   
54699                       @Jessicabmoura e a make avon       en   
8351                              Where do u live 😂😂😂😂😂😂       en   
9910                  @FNJacko @cxlEU your mum does avon       en   
15851                                           @O2 Done       en   
16523                                 @MAK_AVON O Hairrr       en   
21298                            @dmsreadwrite We do! 😊😊       en   

                                                hashtags  day hour  nlikes  \
3263                                                  []    1   06       0   
3654                                                  []    1   02       1   
3895                                                  []    1   00       1   
13210                                                 []    5   00       1   
14083                                                 []    4   17       1   
18139                                                 []    3   09       4   
19647                                                 []    2   19       1   
19936                                                 []    2   17       0   
20146                                                 []    2   15       1   
26522                                                 []    7   08       0   
32900                                                 []    4   19       0   
33513                                                 []    4 

De todos os tweets marcados como EN e detectados como PT somente um aparenta ter relevância para a análise e será incluído no dataset final. ID: 45671

In [138]:
en_tweets.loc[45671,:]

id                                             1354512074416812035
date                                           2021-01-27 16:30:18
place                                                             
tweet            @vaidesmaiar Avon queria Bubu MakeUp e ganhou ...
language                                                        en
hashtags                                          [redebbb, bbb21]
day                                                              3
hour                                                            16
nlikes                                                          18
nreplies                                                         0
nretweets                                                        2
search                                                        Avon
pos_bbb                                                          1
detected_lang                                                   pt
Name: 45671, dtype: object

In [143]:
pt_tweets = ft_tweets.query("language == 'pt'")
pt_tweets = pt_tweets.append(en_tweets.loc[45671,:])

#### Verificação do total de dias que restaram após a remoção dos tweets em inglês.

In [148]:
pt_tweets['date'] = pd.to_datetime(pt_tweets['date'])

In [157]:
print(f"Total Dias Pós: {pt_tweets.query('pos_bbb == 1')['date'].max() - pt_tweets.query('pos_bbb == 1')['date'].min()}")
print(f"Total Dias Pré: {pt_tweets.query('pos_bbb == 0')['date'].max() - pt_tweets.query('pos_bbb == 0')['date'].min()}")

Total Dias Pós: 14 days 23:52:49
Total Dias Pré: 14 days 23:51:12


#### Após as elimitações dos tweets detectados em outros idiomas, vamos verificar quantos tweets temos nos períodos antes e após o início do programa.

In [147]:
pt_tweets.groupby('pos_bbb').agg(
    total=('id','count')
)

total
pos_bbb       
0         2838
1        25306

#### Ao longo dos 14 dias antes do início do programa e nos 14 dias após, percebemos um grande aumento no volume de Tweets para a Avon, vamos explorar um pouco mais estes dados.

### 3 - Análise temporal

In [163]:
pt_tweets['dates'] = pt_tweets['date'].dt.date 

In [165]:
tweets_day = pt_tweets.groupby('dates').agg(
    tweets_dia=('id','count')
)

In [167]:
px.line(tweets_day)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed